In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or '2' to hide INFO messages as well

# Optionally, set TensorFlow's own logger level to ERROR
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

# For Transformers, reduce its verbosity:
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

# Optionally, ignore Python warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
import tensorflow as tf
from transformers import AutoTokenizer

In [3]:
model_loc = "saved_model/xlm-r_94acc_best_model_v3"
# model_loc = "saved_model/xmlr-stff-best_fix_v2"

model_name = "xlm-roberta-base"


In [4]:
input1 = "hahaha kidding aside, I think  give chance to others na, especially sa mga aspiring leaders natin here ðŸ˜‰. Why not itry niyo rin promise enjoy yan âœ¨ we, mga kuyas and ates will follow the lead"
input2 = "Shopee 80% OFF! Order now: shopee-80store.com. Last day na!"
input3 = "Di ako pwede lumabas ngayon may ginagawa pa ako baka next time na lang"
input4 = "PLDT detected multiple login attempts sa ibang bansa! Secure account now! pldt-fix.ml"
input5 = "Hungry for discounts? As a new Smart subscriber, we have an exclusive foodpanda voucher just for you!1. Open the foodpanda app now: https://prf.hn/l/MbnvkdO2. Enter this code upon checkout: SPPXFPNEW Voucher is valid until January 31, 2025 only! DTI209357"
input_text = input1
input_text = "Introducing the latest 140-hectare green city business district & township development in the South, the MAPLE GROVE. \n COMMERCIAL LOTS are up for sale at this budding Makati-inspired business district in General Trias, Cavite - only 20 minutes away from Mall of Asia and Entertainment City via CAVITEX. \n Lots from 439 to 1,750 sqm are available. Office buildings, business hotels, medical facilities, etc. may be built on these lots. \n Avail now at pre-selling prices and be one of the few owners of commercial lots at the FUTURE CENTRAL BUSINESS DISTRICT this country's biggest developers are developing. \n For a presentation on the project, please contact our sales manager, Ms. Beatrice, at 091165210150. \n Megaworld Corp."

In [5]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("saved_tokenizer/" + "for_xmlr-pd-best_fix_v1")

In [6]:
# Tokenize the input text
encoded_inputs = tokenizer(input_text, return_tensors="tf", padding=True, truncation=True)

# dummy token_type_ids for tokenizing
dummy_token_type_ids = tf.zeros_like(encoded_inputs["input_ids"])

# Prepare the input dictionary
inputs = {
    "input_ids": encoded_inputs["input_ids"],
    "attention_mask": encoded_inputs["attention_mask"],
    "token_type_ids": dummy_token_type_ids,
}

# Load the SavedModel
loaded_model = tf.saved_model.load(model_loc)
infer = loaded_model.signatures["serving_default"]

# Run inference
outputs = infer(**inputs)

# Extract logits
logits = outputs["logits"]

# Apply softmax to get probabilities
probabilities = tf.nn.softmax(logits)

# Get the predicted class index
predicted_class_index = tf.argmax(probabilities, axis=1).numpy()[0]

# Map class index to label
class_labels = {0: "HAM", 1: "SCAM"}
predicted_label = class_labels[predicted_class_index]

# Output the result
print("Message: " + input_text)
print(f"The message is classified as: {predicted_label}")

Message: Introducing the latest 140-hectare green city business district & township development in the South, the MAPLE GROVE. 
 COMMERCIAL LOTS are up for sale at this budding Makati-inspired business district in General Trias, Cavite - only 20 minutes away from Mall of Asia and Entertainment City via CAVITEX. 
 Lots from 439 to 1,750 sqm are available. Office buildings, business hotels, medical facilities, etc. may be built on these lots. 
 Avail now at pre-selling prices and be one of the few owners of commercial lots at the FUTURE CENTRAL BUSINESS DISTRICT this country's biggest developers are developing. 
 For a presentation on the project, please contact our sales manager, Ms. Beatrice, at 091165210150. 
 Megaworld Corp.
The message is classified as: HAM


In [7]:
probabilities

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.58734375, 0.41265628]], dtype=float32)>